In [1]:
import numpy as np
import pandas as pd

In [2]:
#import csv file with columns and turn into DataFrame
mathands = pd.read_csv("mathands_compiled_m8_columns.csv")
mathands_df = pd.DataFrame(mathands)
mathands_df

,Begin Time - ss.msec,End Time - ss.msec,Duration - ss.msec,*TIME,LHGrasp,RHInteract,RHGrasp,Interact Comment,LHInteract,Grasp Comment,LHinteract,File,File Path
0,119.006,119.382,0.376,START,NaN,NaN,NaN,NaN,NaN,NaN,NaN,001_08_071807_07172018EP_mathands_KC.eaf,NaN
1,119.006,138.795,19.789,NaN,NaN,NaN,mg,NaN,NaN,NaN,NaN,001_08_071807_07172018EP_mathands_KC.eaf,NaN
2,119.006,138.796,19.790,NaN,NaN,m,NaN,NaN,NaN,NaN,NaN,001_08_071807_07172018EP_mathands_KC.eaf,NaN
3,137.203,137.597,0.394,NaN,mg,NaN,NaN,NaN,NaN,NaN,NaN,001_08_071807_07172018EP_mathands_KC.eaf,NaN
4,137.204,137.596,0.392,NaN,NaN,NaN,NaN,NaN,m,NaN,NaN,001_08_071807_07172018EP_mathands_KC.eaf,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2530,390.126,390.392,0.266,NaN,NaN,m,NaN,NaN,NaN,NaN,NaN,048_08_020409MA_SYNC_CLEAN_mathands_DH.eaf,NaN
2531,394.308,399.769,5.461,NaN,NaN,m,NaN,NaN,NaN,NaN,NaN,048_08_020409MA_SYNC_CLEAN_mathands_DH.eaf,NaN
2532,394.407,399.781,5.374,NaN,NaN,NaN,mg,NaN,NaN,NaN,NaN,048_08_020409MA_SYNC_CLEAN_mathands_DH.eaf,NaN
2533,404.946,405.081,0.135,End Toy Play - by BK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,048_08_020409MA_SYNC_CLEAN_mathands_DH.eaf,NaN


In [3]:
#keep time, interact and file name columns
def filter_columns(data):
    keep_columns = ['Begin Time - ss.msec', 'End Time - ss.msec', 'RHInteract','LHInteract','LHinteract','File']
    filtered_data = data[keep_columns]
    return filtered_data

#rename columns
def rename_columns(data, column_names):
    renamed_data = data.rename(columns=column_names)
    return renamed_data

column_names = {
    'Begin Time - ss.msec': "Begin",
    'End Time - ss.msec': "End",
}

#rename the file names into just ID and month (i.e.'001_08')
def file_rename(data, column_name):
    data[column_name] = data[column_name].str[:6]
    return data

#apply above functions
mathands_df = filter_columns(mathands_df)
mathands_df = rename_columns(mathands_df, column_names)
mathand_df = file_rename(mathands_df, "File")


In [4]:
#remove all nan values and only keep m, i, and a
def remove_nan(data):
    modified_data = data.fillna('')
    return modified_data     

#since there are two LH interact columns, move vaue from LHinteract to LHIneract  
def combine_LHInteract(data, column_a, column_b):
    data[column_a] = data.apply(lambda row: row[column_b] if row[column_b] != '' else row[column_a], axis=1)
    data.drop(columns=[column_b], inplace=True)
    return data

#apply above functions
mathands_df = remove_nan(mathands_df)
mathands_df = combine_LHInteract(mathands_df,"LHInteract", "LHinteract")



In [5]:
#create musical toy column
def musical_column(data, column_a, column_b):
    data['m'] = ''
    for index, value in data[column_a].iteritems():
        if value == 'm':
            data.at[index, 'm'] = 'R'
            
    for index, value in data[column_b].iteritems():
        if value == 'm':
            data.at[index, 'm'] = "L"
    return data

mathands_df = musical_column(mathands_df,'RHInteract',"LHInteract")

#create animate toy column 
def animate_column(data, column_a, column_b):
    data['a'] = ''
    for index, value in data[column_a].iteritems():
        if value == 'a':
            data.at[index, 'a'] = "R"
            
    for index, value in data[column_b].iteritems():
        if value == 'a':
            data.at[index, 'a'] = "L"
    return data

mathands_df = animate_column(mathands_df,'RHInteract',"LHInteract")

#create inanimate toy column 
def inanimate_column(data, column_a, column_b):
    data['i'] = ''
    for index, value in data[column_a].iteritems():
        if value == 'i':
            data.at[index, 'i'] = 'R'
            
    for index, value in data[column_b].iteritems():
        if value == 'i':
            data.at[index, 'i'] = 'L'
    return data

mathands_df = inanimate_column(mathands_df,'RHInteract',"LHInteract")



In [6]:
def drop_empty_rows(data, column_a, column_b):
    data.drop(data[(data[column_a] == '') & (data[column_b] == '')].index, inplace=True)
    return data

mathands_df = drop_empty_rows(mathands_df,'RHInteract',"LHInteract")
mathands_df = mathands_df.drop(labels=['RHInteract','LHInteract'], axis=1)

#reindex to make sure order is corrrect 
columns_order = ['Begin', 'End','m','i','a','File']
mathands_df = mathands_df.reindex(columns=columns_order)

#reset index from 0 without jumps
mathands_df = mathands_df.reset_index()
mathands_df = mathands_df.drop(labels=['index'], axis=1)
mathands_df = mathands_df.reset_index()

In [7]:
mathands_df[0:20]

,index,Begin,End,m,i,a,File
0,0,119.006,138.796,R,,,001_08
1,1,137.204,137.596,L,,,001_08
2,2,147.201,148.199,,L,,001_08
3,3,147.204,148.594,,R,,001_08
4,4,151.001,152.494,R,,,001_08
5,5,153.901,165.594,,R,,001_08
6,6,156.503,161.496,,L,,001_08
7,7,161.901,164.498,,,L,001_08
8,8,165.205,166.799,,L,,001_08
9,9,182.703,184.996,,L,,001_08


In [8]:
#If Being and End times overlap, change value to Both
# create an empty dataframe to store the new rows
new_data = pd.DataFrame(columns=mathands_df.columns)

groups = mathands_df.groupby('File')

# loop through each row in the original data
for i, row in mathands_df.iterrows():
    # check if there is any overlap with the previous row
    if i > 0 and row['File'] == mathands_df.loc[i-1, 'File'] and row['Begin'] < mathands_df.loc[i-1, 'End']:
        # if there is overlap, add a new row to the new_data dataframe with "both" as the value for the corresponding columns
        overlap_start = max(row['Begin'], mathands_df.loc[i-1, 'Begin'])
        overlap_end = min(row['End'], mathands_df.loc[i-1, 'End'])
        new_row = row.copy()
        new_row['Begin'] = overlap_start
        new_row['End'] = overlap_end
        new_row[new_row == 'R'] = 'Both'
        new_row[new_row == 'L'] = 'Both'
        new_data = new_data.append(new_row, ignore_index=True)
        
    # add the original row to the new_data dataframe
    new_data = new_data.append(row, ignore_index=True)


In [9]:
new_data

,index,Begin,End,m,i,a,File
0,0,119.006,138.796,R,,,001_08
1,1,137.204,137.596,Both,,,001_08
2,1,137.204,137.596,L,,,001_08
3,2,147.201,148.199,,L,,001_08
4,3,147.204,148.199,,Both,,001_08
...,...,...,...,...,...,...,...
2002,1541,381.205,381.973,R,,,048_08
2003,1542,382.610,386.782,L,,,048_08
2004,1543,389.627,389.795,L,,,048_08
2005,1544,390.126,390.392,R,,,048_08


In [10]:
#delete any duplicated rows with the same Begin and End time 
grouped = new_data.groupby('File')

# define a function to apply to each group
def drop_duplicates(group):
    # check for rows with the same begin and end time
    duplicates = group.duplicated(subset=['Begin', 'End'])
    # drop the duplicates
    group = group[~duplicates].drop_duplicates()
    return group

# apply the function to each group
df_cleaned = grouped.apply(drop_duplicates)
df_cleaned = df_cleaned.droplevel('File')

In [11]:
#update times to reflect the overlapped times  
grouped = df_cleaned.groupby('File')

def update_times(group):
    # sort the group by begin time
    group = group.sort_values(by='Begin')
    
    # initialize previous row as the first row of the group
    prev_row = group.iloc[0]
    new_rows = [] 
    
    # iterate through each row
    for i in range(1, len(group)):
        row = group.iloc[i]
        
        # if the end time of the current row is the same as the end time of the previous row
        if row['End'] == prev_row['End']:
            # update the end time of the previous row to be the start time of the current row
            group.loc[prev_row.name, 'End'] = row['Begin']
        
        # if the start time of the current row is the same as the start time of the previous row
        if row['Begin'] == prev_row['Begin']:
            # update the start time of the current row to be the end time of the previous row
            group.loc[row.name, 'Begin'] = prev_row['End']

    
        elif row['Begin'] > prev_row['Begin'] and row['End'] < prev_row['End']:
            group.loc[prev_row.name, 'End'] = row['Begin']
            # create a new row with start time as the end time of the current row
            # and end time as the end time of the previous row
            new_row = {
                'Begin': row['End'],
                'End': prev_row['End'],
                'm': prev_row['m'],
                'i': prev_row['i'],
                'a': prev_row['a'],                
                'File': prev_row['File']
            }
            # add the new row to the list of new rows
            new_rows.append(new_row)
        
                    
        # set the previous row to the current row for the next iteration
        prev_row = row
        new_df = pd.DataFrame(new_rows)
        group = pd.concat([group, new_df], ignore_index=True)
            
    return group

updated_df = grouped.apply(update_times)


In [12]:
updated_df = updated_df.droplevel('File')

In [13]:
#update times to reflect nest times 
grouped = updated_df.groupby('File')

def nested_times(group):
    # sort the group by begin time
    group = group.sort_values(by='Begin')
    
    # initialize previous row as the first row of the group
    prev_row = group.iloc[0]
    
    # to store new rows to be added
    new_rows = [] 
    
    if row['Begin'] > prev_row['Begin'] and row['End'] < prev_row['End']:
        group.loc[prev_row.name, 'End'] = row['Begin']
            # create a new row with start time as the end time of the current row
            # and end time as the end time of the previous row
        new_row = {
            'Begin': row['End'],
            'End': prev_row['End'],
            'm': prev_row['m'],
            'i': prev_row['i'],
            'a': prev_row['a'],                
            'File': prev_row['File']
        }
            # add the new row to the list of new rows
        new_rows.append(new_row)
            
            
    # set the previous row to the current row for the next iteration
    prev_row = row
        
    new_df = pd.DataFrame(new_rows)
    group = pd.concat([group, new_df], ignore_index=True)
        
    return group

nested_df = grouped.apply(nested_times)

#delete any duplicated rows 
nested_df = nested_df.droplevel('File')
grouped = nested_df.groupby('File')

# define a function to apply to each group
def drop_duplicates(group):
    # check for rows with the same begin and end time
    duplicates = group.duplicated(subset=['Begin', 'End'])
    # drop the duplicates
    group = group[~duplicates].drop_duplicates()
    return group

# apply the function to each group
mathands_df = grouped.apply(drop_duplicates)


In [14]:
mathands_df = mathands_df.droplevel('File')


In [15]:
mathands_df

,index,Begin,End,m,i,a,File
0,0,119.006,137.204,R,,,001_08
1,1,137.204,137.596,Both,,,001_08
2,NaN,137.596,138.796,R,,,001_08
38,2,147.201,147.204,,L,,001_08
39,3,147.204,148.199,,Both,,001_08
...,...,...,...,...,...,...,...
200,1541,381.205,381.973,R,,,048_08
201,1542,382.610,386.782,L,,,048_08
202,1543,389.627,389.795,L,,,048_08
203,1544,390.126,390.392,R,,,048_08


In [16]:
#import csv file with columns and turn into DataFrame
babyhands = pd.read_csv("babyhands_compiled_m8_columns.csv")
babyhands_df = pd.DataFrame(babyhands)

In [17]:
babyhands_df

,Begin Time - ss.msec,End Time - ss.msec,*TIME,Motion,LHGrasp,RHInteract,Mouthing,RHGrasp,Interact Comment,LHInteract,Mom Action,Grasp Comment,Mouth,Mom Actions,LHinteract,Mom Grab,File,File Path
0,119.000,142.899,NaN,NaN,NaN,i,NaN,ig,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,001_08_071807_07172018EP_babyhands_TD.eaf,NaN
1,119.006,119.382,START,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,001_08_071807_07172018EP_babyhands_TD.eaf,NaN
2,123.240,139.679,NaN,NaN,NaN,NaN,NaN,NaN,NaN,i,NaN,NaN,NaN,NaN,NaN,NaN,001_08_071807_07172018EP_babyhands_TD.eaf,NaN
3,123.557,139.679,NaN,NaN,ig,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,001_08_071807_07172018EP_babyhands_TD.eaf,NaN
4,124.417,130.089,NaN,NaN,NaN,NaN,im,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,001_08_071807_07172018EP_babyhands_TD.eaf,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3370,394.459,394.717,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,momgrab,NaN,NaN,048_08_020409MA_SYNC_CLEAN_babyhands_TD.eaf,NaN
3371,400.240,406.797,NaN,NaN,NaN,m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,048_08_020409MA_SYNC_CLEAN_babyhands_TD.eaf,NaN
3372,401.167,403.871,NaN,shak,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,048_08_020409MA_SYNC_CLEAN_babyhands_TD.eaf,NaN
3373,404.946,405.081,End Toy Play - by BK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,048_08_020409MA_SYNC_CLEAN_babyhands_TD.eaf,NaN


In [18]:
#keep only time, interact, mom action, and file columns
babyhands_df = babyhands_df[['Begin Time - ss.msec','End Time - ss.msec','RHInteract','LHInteract', 'LHinteract',
                             'Mom Action','Mom Actions', "Mom Grab",'File']]

#rename columns
babyhands_df = babyhands_df.rename(columns={'Begin Time - ss.msec':'Begin'})
babyhands_df = babyhands_df.rename(columns={'End Time - ss.msec':'End'})

#rename the file names into just ID and month (i.e.'001_08')
for i in babyhands_df['File']:
    newfilename = i[0:6]
    babyhands_df['File'] = babyhands_df['File'].replace(i,newfilename)

In [19]:
#turn all contents in both Interact column and Mom columns into string
babyhands_df[['RHInteract', 'LHInteract','LHinteract','Mom Action','Mom Actions','Mom Grab']] = babyhands_df[['RHInteract', 'LHInteract','LHinteract','Mom Action','Mom Actions','Mom Grab']].astype(str) 

#keep only 'm',' a', 'i' in Interact columns 
for i in babyhands_df['RHInteract']:
    if i == 'nan':
        no_interact = ''
        babyhands_df['RHInteract'] = babyhands_df['RHInteract'].replace(i,no_interact)  

for i in babyhands_df['LHInteract']:
    if i == 'nan':
        no_interact = ''
        babyhands_df['LHInteract'] = babyhands_df['LHInteract'].replace(i,no_interact)  
        
#there are two left hand interact colums
#move all non empty values in LHinteract into LHInteract and remove LHinteract
for index,row in babyhands_df.iterrows():
    if row['LHinteract'] != 'nan':
        babyhands_df.at[index,'LHInteract']= row['LHinteract']
    else:
        continue
babyhands_df = babyhands_df.drop(['LHinteract'], axis=1)
        
#there are three Mom columns
#move all non empty values in Mom Action and Mom grab into Mom Actions and only keep Mom Actions
for index,row in babyhands_df.iterrows():
    if row['Mom Action'] != 'nan':
        babyhands_df.at[index,'Mom Actions']= row['Mom Action']
    else:
        continue
babyhands_df = babyhands_df.drop(['Mom Action'], axis=1)

for index,row in babyhands_df.iterrows():        
    if row['Mom Grab'] != 'nan':
        babyhands_df.at[index,'Mom Actions']= row['Mom Grab']
    else:
        continue
babyhands_df = babyhands_df.drop(['Mom Grab'], axis=1)

for i in babyhands_df['Mom Actions']:
    if i == 'nan':
        no_interact = ''
        babyhands_df['Mom Actions'] = babyhands_df['Mom Actions'].replace(i,no_interact)  
        

In [20]:
#create new columns for three toys
#iterate through every row for each toy type both RHInteract and LHInteract

#musical toy
babyhands_df['m'] = ""
for index,row in babyhands_df.iterrows():
    if row['RHInteract'] == 'm':
        babyhands_df.at[index,'m']= 'R'
        
for index,row in babyhands_df.iterrows():
    if row['LHInteract'] == 'm':
        babyhands_df.at[index,'m']= 'L'

#inanimate toy 
babyhands_df['i'] = ""
for index,row in babyhands_df.iterrows():
    if row['RHInteract'] == 'i':
        babyhands_df.at[index,'i']= 'R'
        
for index,row in babyhands_df.iterrows():
    if row['LHInteract'] == 'i':
        babyhands_df.at[index,'i']= 'L'

#animate toy  
babyhands_df['a'] = ""
for index,row in babyhands_df.iterrows():
    if row['RHInteract'] == 'a':
        babyhands_df.at[index,'a']= 'R'

for index,row in babyhands_df.iterrows():
    if row['LHInteract'] == 'a':
        babyhands_df.at[index,'a']= 'L'

In [21]:
#only keep time, toy types, and file columns
#reindex to make sure order is corrrect 
babyhands_df = babyhands_df.drop(labels=['RHInteract','LHInteract'], axis=1)
columns_order = ['Begin', 'End','m','i','a','Mom Actions','File']
babyhands_df = babyhands_df.reindex(columns=columns_order)

In [22]:
#drop rows with no values
for index,row in babyhands_df.iterrows():
    if row['m']=='':
        if row['i']=='':
            if row['a']=='':
                if row['Mom Actions']=='':
                    babyhands_df = babyhands_df.drop(index)
                
#reset index from 0 without jumps
babyhands_df = babyhands_df.reset_index()
babyhands_df = babyhands_df.drop(labels=['index'], axis=1)
babyhands_df = babyhands_df.reset_index()
         


In [23]:
babyhands_df[0:20]

,index,Begin,End,m,i,a,Mom Actions,File
0,0,119.000,142.899,,R,,,001_08
1,1,123.240,139.679,,L,,,001_08
2,2,139.613,140.573,,,,momgrab,001_08
3,3,140.046,144.592,L,,,,001_08
4,4,142.913,149.789,R,,,,001_08
5,5,145.356,155.489,,L,,,001_08
6,6,149.789,159.486,,R,,,001_08
7,7,154.430,162.023,,,,momgrab,001_08
8,8,159.102,164.188,,L,,,001_08
9,9,161.573,177.493,,,R,,001_08


In [24]:
#If Being and End times overlap, change value to Both
# create an empty dataframe to store the new rows
new_data = pd.DataFrame(columns=babyhands_df.columns)

groups = babyhands_df.groupby('File')

# loop through each row in the original data
for i, row in babyhands_df.iterrows():
    # check if there is any overlap with the previous row
    if i > 0 and row['File'] == babyhands_df.loc[i-1, 'File'] and row['Begin'] < babyhands_df.loc[i-1, 'End']:
        # if there is overlap, add a new row to the new_data dataframe with "both" as the value for the corresponding columns
        overlap_start = max(row['Begin'], babyhands_df.loc[i-1, 'Begin'])
        overlap_end = min(row['End'], babyhands_df.loc[i-1, 'End'])
        new_row = row.copy()
        new_row['Begin'] = overlap_start
        new_row['End'] = overlap_end
        new_row[new_row == 'R'] = 'Both'
        new_row[new_row == 'L'] = 'Both'
        new_data = new_data.append(new_row, ignore_index=True)
        
    # add the original row to the new_data dataframe
    new_data = new_data.append(row, ignore_index=True)




In [25]:
new_data

,index,Begin,End,m,i,a,Mom Actions,File
0,0,119.000,142.899,,R,,,001_08
1,1,123.240,139.679,,Both,,,001_08
2,1,123.240,139.679,,L,,,001_08
3,2,139.613,139.679,,,,momgrab,001_08
4,2,139.613,140.573,,,,momgrab,001_08
...,...,...,...,...,...,...,...,...
2235,1551,389.063,394.694,R,,,,048_08
2236,1552,391.466,391.782,Both,,,,048_08
2237,1552,391.466,391.782,L,,,,048_08
2238,1553,394.459,394.717,,,,momgrab,048_08


In [26]:
#delete any duplicated rows with the same Begin and End time 
grouped = new_data.groupby('File')

# define a function to apply to each group
def drop_duplicates(group):
    # check for rows with the same begin and end time
    duplicates = group.duplicated(subset=['Begin', 'End'])
    # drop the duplicates
    group = group[~duplicates].drop_duplicates()
    return group

# apply the function to each group
df_cleaned = grouped.apply(drop_duplicates)
df_cleaned = df_cleaned.droplevel('File')

In [27]:
df_cleaned[0:20]

,index,Begin,End,m,i,a,Mom Actions,File
0,0,119.000,142.899,,R,,,001_08
1,1,123.240,139.679,,Both,,,001_08
3,2,139.613,139.679,,,,momgrab,001_08
4,2,139.613,140.573,,,,momgrab,001_08
5,3,140.046,140.573,Both,,,,001_08
6,3,140.046,144.592,L,,,,001_08
7,4,142.913,144.592,Both,,,,001_08
8,4,142.913,149.789,R,,,,001_08
9,5,145.356,149.789,,Both,,,001_08
10,5,145.356,155.489,,L,,,001_08


In [28]:
#update times to reflect the overlapped times  
grouped = df_cleaned.groupby('File')

def update_times(group):
    # sort the group by begin time
    group = group.sort_values(by='Begin')
    
    # initialize previous row as the first row of the group
    prev_row = group.iloc[0]
    new_rows = [] 
    
    # iterate through each row
    for i in range(1, len(group)):
        row = group.iloc[i]
        
        # if the end time of the current row is the same as the end time of the previous row
        if row['End'] == prev_row['End']:
            # update the end time of the previous row to be the start time of the current row
            group.loc[prev_row.name, 'End'] = row['Begin']
        
        # if the start time of the current row is the same as the start time of the previous row
        if row['Begin'] == prev_row['Begin']:
            # update the start time of the current row to be the end time of the previous row
            group.loc[row.name, 'Begin'] = prev_row['End']
            
        if row['Begin'] == prev_row['End']:
            # drop the current row
            group = group.drop(row.name)
            # update the end time of the previous row to be the end time of the current row
            group.loc[prev_row.name, 'End'] = row['End']
            
        elif row['Begin'] > prev_row['Begin'] and row['End'] < prev_row['End']:
            group.loc[prev_row.name, 'End'] = row['Begin']
            # create a new row with start time as the end time of the current row
            # and end time as the end time of the previous row
            new_row = {
                'Begin': row['End'],
                'End': prev_row['End'],
                'm': prev_row['m'],
                'i': prev_row['i'],
                'a': prev_row['a'],  
                'Mom Actions':prev_row['Mom Actions'],
                'File': prev_row['File']
            }
            # add the new row to the list of new rows
            new_rows.append(new_row)
        
                    
        # set the previous row to the current row for the next iteration
        prev_row = row
        new_df = pd.DataFrame(new_rows)
        group = pd.concat([group, new_df], ignore_index=True)
            
    return group

updated_df = grouped.apply(update_times)


In [29]:
updated_df = updated_df.droplevel('File')

In [30]:
updated_df[0:20]

,index,Begin,End,m,i,a,Mom Actions,File
0,0,119.000,123.240,,R,,,001_08
1,1,123.240,139.613,,Both,,,001_08
2,2,139.613,139.679,,,,momgrab,001_08
3,2,139.679,140.046,,,,momgrab,001_08
4,3,140.046,140.573,Both,,,,001_08
5,3,140.573,142.913,L,,,,001_08
6,4,142.913,144.592,Both,,,,001_08
7,4,144.592,145.356,R,,,,001_08
8,5,145.356,149.789,,Both,,,001_08
9,5,149.789,149.789,,L,,,001_08


In [31]:
#update times to reflect nest times 
grouped = updated_df.groupby('File')

def nested_times(group):
    # sort the group by begin time
    group = group.sort_values(by='Begin')
    
    # initialize previous row as the first row of the group
    prev_row = group.iloc[0]
    
    # to store new rows to be added
    new_rows = [] 
    
    if row['Begin'] > prev_row['Begin'] and row['End'] < prev_row['End']:
        group.loc[prev_row.name, 'End'] = row['Begin']
            # create a new row with start time as the end time of the current row
            # and end time as the end time of the previous row
        new_row = {
            'Begin': row['End'],
            'End': prev_row['End'],
            'm': prev_row['m'],
            'i': prev_row['i'],
            'a': prev_row['a'],                
            'File': prev_row['File']
        }
            # add the new row to the list of new rows
        new_rows.append(new_row)
            
            
    # set the previous row to the current row for the next iteration
    prev_row = row
        
    new_df = pd.DataFrame(new_rows)
    group = pd.concat([group, new_df], ignore_index=True)
        
    return group

nested_df = grouped.apply(nested_times)

#delete any duplicated rows 
nested_df = nested_df.droplevel('File')
grouped = nested_df.groupby('File')

# define a function to apply to each group
def drop_duplicates(group):
    # check for rows with the same begin and end time
    duplicates = group.duplicated(subset=['Begin', 'End'])
    # drop the duplicates
    group = group[~duplicates].drop_duplicates()
    return group

# apply the function to each group
babyhands_df = grouped.apply(drop_duplicates)
babyhands_df = babyhands_df.droplevel('File')


In [32]:
babyhands_df

,index,Begin,End,m,i,a,Mom Actions,File
0,0,119.000,123.240,,R,,,001_08
1,1,123.240,139.613,,Both,,,001_08
2,2,139.613,139.679,,,,momgrab,001_08
3,NaN,139.679,142.899,,R,,,001_08
57,2,139.679,140.046,,,,momgrab,001_08
...,...,...,...,...,...,...,...,...
813,1551,389.627,391.466,R,,,,048_08
814,1552,391.466,391.782,Both,,,,048_08
815,NaN,391.782,394.694,R,,,,048_08
818,1553,394.459,394.717,,,,momgrab,048_08


In [33]:
#merge babyhands and mathands
matbaby_df = pd.merge(babyhands_df, mathands_df, how='outer', on=['Begin','End'],
                     suffixes = ('_inf','_mom'))


In [34]:
#change all contents in string 
matbaby_df[['m_inf', 'i_inf','a_inf','Mom Actions','File_inf',
                'm_mom', 'i_mom', 'a_mom','File_mom']] = matbaby_df[['m_inf', 'i_inf','a_inf','Mom Actions','File_inf',
                'm_mom', 'i_mom', 'a_mom','File_mom']].astype(str)

In [35]:
#correspond file name of mom and inf to each other
for index,row in matbaby_df.iterrows():
    if row['File_mom'] == 'nan':
        matbaby_df.at[index,'File_mom']= row['File_inf']
    else:
        continue
        
for index,row in matbaby_df.iterrows():   
    if row['File_inf'] == 'nan':
        matbaby_df.at[index,'File_inf']= row['File_mom']
    else:
        continue

#check file names are the same
#if they are, keep only one and rename to just File
for index,row in matbaby_df.iterrows():   
    if row['File_inf'] != row['File_mom']:
        print('file name not the same')
    else:
        continue   
        
matbaby_df = matbaby_df.drop('File_inf', axis=1)
matbaby_df = matbaby_df.rename(columns={'File_mom':'File'})
        

In [36]:
matbaby_df

,index_inf,Begin,End,m_inf,i_inf,a_inf,Mom Actions,index_mom,m_mom,i_mom,a_mom,File
0,0,119.000,123.240,,R,,,NaN,nan,nan,nan,001_08
1,1,123.240,139.613,,Both,,,NaN,nan,nan,nan,001_08
2,2,139.613,139.679,,,,momgrab,NaN,nan,nan,nan,001_08
3,NaN,139.679,142.899,,R,,,NaN,nan,nan,nan,001_08
4,2,139.679,140.046,,,,momgrab,NaN,nan,nan,nan,001_08
...,...,...,...,...,...,...,...,...,...,...,...,...
4196,NaN,381.205,381.973,nan,nan,nan,nan,1541,R,,,048_08
4197,NaN,382.610,386.782,nan,nan,nan,nan,1542,L,,,048_08
4198,NaN,389.627,389.795,nan,nan,nan,nan,1543,L,,,048_08
4199,NaN,390.126,390.392,nan,nan,nan,nan,1544,R,,,048_08


In [37]:
#turn all nan values into empty strings
for index,row in matbaby_df.iterrows():
    if row['m_inf'] == 'nan':
        matbaby_df.at[index,'m_inf']= ''
    else:
        continue

for index,row in matbaby_df.iterrows():
    if row['i_inf'] == 'nan':
        matbaby_df.at[index,'i_inf']= ''
    else:
        continue

for index,row in matbaby_df.iterrows():
    if row['a_inf'] == 'nan':
        matbaby_df.at[index,'a_inf']= ''
    else:
        continue
        
for index,row in matbaby_df.iterrows():
    if row['m_mom'] == 'nan':
        matbaby_df.at[index,'m_mom']= ''
    else:
        continue

for index,row in matbaby_df.iterrows():
    if row['i_mom'] == 'nan':
        matbaby_df.at[index,'i_mom']= ''
    else:
        continue

for index,row in matbaby_df.iterrows():
    if row['a_mom'] == 'nan':
        matbaby_df.at[index,'a_mom']= ''
    else:
        continue

for index,row in matbaby_df.iterrows():
    if row['Mom Actions'] == 'nan':
        matbaby_df.at[index,'Mom Actions']= ''
    else:
        continue

In [38]:
#delete rows where interact is empty
for index,row in matbaby_df.iterrows():
    if row['m_inf']=='':
        if row['i_inf']=='':
            if row['a_inf']=='':
                if row['Mom Actions']=='':
                    if row['m_mom']=='':
                        if row['i_mom']=='':
                            if row['a_mom']=='':
                                matbaby_df = matbaby_df.drop(index)
                                
                                
                                
#reset index from 0 without jumps
matbaby_df = matbaby_df.reset_index()
matbaby_df = matbaby_df.drop(labels=['index'], axis=1)

In [39]:
matbaby_df

,index_inf,Begin,End,m_inf,i_inf,a_inf,Mom Actions,index_mom,m_mom,i_mom,a_mom,File
0,0,119.000,123.240,,R,,,NaN,,,,001_08
1,1,123.240,139.613,,Both,,,NaN,,,,001_08
2,2,139.613,139.679,,,,momgrab,NaN,,,,001_08
3,NaN,139.679,142.899,,R,,,NaN,,,,001_08
4,2,139.679,140.046,,,,momgrab,NaN,,,,001_08
...,...,...,...,...,...,...,...,...,...,...,...,...
4149,NaN,381.205,381.973,,,,,1541,R,,,048_08
4150,NaN,382.610,386.782,,,,,1542,L,,,048_08
4151,NaN,389.627,389.795,,,,,1543,L,,,048_08
4152,NaN,390.126,390.392,,,,,1544,R,,,048_08


In [40]:
matbaby_df = matbaby_df.drop(labels=['index_mom'], axis=1)
matbaby_df = matbaby_df.drop(labels=['index_inf'], axis=1)

In [41]:
matbaby_df

,Begin,End,m_inf,i_inf,a_inf,Mom Actions,m_mom,i_mom,a_mom,File
0,119.000,123.240,,R,,,,,,001_08
1,123.240,139.613,,Both,,,,,,001_08
2,139.613,139.679,,,,momgrab,,,,001_08
3,139.679,142.899,,R,,,,,,001_08
4,139.679,140.046,,,,momgrab,,,,001_08
...,...,...,...,...,...,...,...,...,...,...
4149,381.205,381.973,,,,,R,,,048_08
4150,382.610,386.782,,,,,L,,,048_08
4151,389.627,389.795,,,,,L,,,048_08
4152,390.126,390.392,,,,,R,,,048_08


In [44]:
matbaby_df.to_csv('matbaby.csv', index=False)